In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import json
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from icecream import ic
from import_casa import casa, cano
from casa.annot import AspectEnum

In [36]:
TARGET_DIR = "20210430"
result_dir = casa.get_data_path() / f"caprice/"
result_dir.mkdir(exist_ok=True, parents=True)
json_paths = [x for x in result_dir.iterdir() if x.suffix==".json"]

In [4]:
with json_paths[1].open("r", encoding="UTF-8") as fin:
    annots = json.load(fin)

In [5]:
list(annots[0].keys())

['completions', 'data', 'id']

## Main conversion loop

In [21]:
seq_pairs = []
seq_labels = []
for json_path in tqdm(json_paths):
    with json_path.open("r", encoding="UTF-8") as fin:
        annots = json.load(fin)
    for annot_i, annot_x in enumerate(annots): 
        try:
            aspects = cano.process_thread_annotations(annot_x)
            pairs, labels = cano.make_sequence_for_caprice(aspects, annot_x["data"]["html"], noise_ratio=0.1)
            seq_pairs.extend(pairs)
            seq_labels.extend(labels)            
        except Exception:
            print(annot_i)
            raise Exception()

  0%|                                                                                                       | 0/10 [00:00<?, ?it/s]

different text spans:  方便又迅速！Good!


 40%|██████████████████████████████████████                                                         | 4/10 [00:01<00:02,  2.42it/s]

different text spans:  前18個月319/一個月，後六個月一個月399
different text spans:  中華電信5G有在建設？？出門連到5G的機會還是不到1/10
different text spans:  連玩coin master都會斷線的電信我真的不知道要說什麼
different text spans:  每家都有表現差的狀況，就是看環境適合誰囉，我生活圈台哥就是順
different text spans:  用起來是比其他家都還快
different text spans:  不過遠傳這次拿雙冠真的有讓人刮目相看


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.03it/s]


In [22]:
len(seq_pairs), len(seq_labels)

(2795, 2795)

In [23]:
np.unique(seq_labels, return_counts=True)

(array([0, 1, 2]), array([1148,  841,  806], dtype=int64))

In [37]:
idx = 1100
seq_pairs[idx][0], seq_labels[idx]
seq_data = [(pair[0], label) 
            for pair, label 
            in zip(seq_pairs, seq_labels)]

In [38]:
seq_data[1252]

('原號漫遊還是較方便，到日本我幾乎是辦中華電信，訊號很穩定', 2)

## Saving data

In [39]:
with open(result_dir / f"caprice_seq_data_{TARGET_DIR}.pkl", "wb") as fout:
    pickle.dump((seq_data), fout)

In [41]:
print(result_dir / f"caprice_seq_data_{TARGET_DIR}.pkl")

E:\LangOn\casa\data\caprice\caprice_seq_data_20210430.pkl
